In [ ]:
pip install xlwings

In [ ]:
import xlwings as xw

import pandas as pd
from datetime import datetime, timedelta,time
import requests
from time import sleep

In [ ]:
wb = xw.Book(r'C:\Users\hari\Desktop\demoXlwing.xlsx')    

In [ ]:
sheet = wb.sheets['Sheet1']

In [ ]:
sheet = wb.sheets['Sheet2']

In [ ]:
sheet['A1'].value = 'Foo 1'

In [ ]:
sheet['A1'].value

In [ ]:
df = pd.DataFrame([[1,2], [3,4]], columns=['a', 'b'])
df

In [ ]:
sheet['E6'].value = df

In [ ]:
sheet['E6'].options(pd.DataFrame, expand='table').value

In [ ]:
sheet.clear_contents()

In [ ]:
apikey = 'GOHDKDWV'
username = 'N890646'
pwd = '1111'
token = 'H3CJDUDQRQUSINVQOK473XJKFNA'

In [ ]:
from smartapi import SmartConnect
import pyotp
obj=SmartConnect(api_key=apikey)
data = obj.generateSession(username,pwd,pyotp.TOTP(token).now())
data['data']['exchanges']

In [ ]:
import requests
url = 'https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json'
d = requests.get(url).json()
token_df = pd.DataFrame.from_dict(d)
token_df['expiry'] = pd.to_datetime(token_df['expiry']).apply(lambda x: x.date())
token_df = token_df.astype({'strike': float})
token_df

In [ ]:
eqSym = token_df[(token_df.symbol.str.endswith('EQ'))]
eqSym

In [ ]:
def getData(token,obj):
    historicParam={
            "exchange": "NSE",
            "symboltoken": f'{token}', 
            "interval": "ONE_DAY",
            "fromdate": "2022-10-02 09:15", 
            "todate": f"{datetime.now().date()} 15:30"
            }
            
        
    res_json =obj.getCandleData(historicParam)
    columns = ['timestamp','open','high','low','close','volume']
    df = pd.DataFrame(res_json['data'], columns=columns)
    df['timestamp'] = pd.to_datetime(df['timestamp'],format = '%Y-%m-%dT%H:%M:%S')
    return df

In [ ]:
getData(11536,obj)

In [ ]:
sheet['H5'].value = 'TCS'
sheet['E6'].value =getData(11536,obj).tail(10)

In [ ]:
for i in eqSym.index[:50]:
    token = eqSym.loc[i]['token']
    sheet['H5'].value =  eqSym.loc[i]['symbol']
    sheet['E6'].value = getData(token,obj).tail(10)
    sleep(2)